In [1]:
# Install dependencies

import sys
!{sys.executable} -m pip install hdfs

In [2]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/Habitos_sustentaveis/gold'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

DataFrame[]

In [2]:
from pyspark.sql.functions import substring, avg, sum

# read air_quality from the silver tables
hdfs_path = "hdfs://hdfs-nn:9000/TrabalhoPL/silver/Projeto.db/Tabela_Petroleo"

tabela_petroleo = spark\
             .read\
             .load(hdfs_path)

tabela_petroleo.show()
tabela_petroleo.printSchema()

+------------+---------------------+--------------------+----------------+--------+---------+----------+-------------------+--------------------+--------------------+--------------------+------------------+--------+------------+---------+------------+-------------+----------+----+
|Spill_Number|Program_Facility_Name|              Street|        Locality|  Contry|SWIS_Code|DEC_Region|Contributing_Factor|           Waterbody|              Source|       Material_Name|   Material_Family|Quantity|       Units|Recovered|Data_Derrame|Data_Relatada|  Data_Fim| Ano|
+------------+---------------------+--------------------+----------------+--------+---------+----------+-------------------+--------------------+--------------------+--------------------+------------------+--------+------------+---------+------------+-------------+----------+----+
|    9506819 | 1 BEEKMAN PLACE  ...|     1 BEEKMAN PLACE|       MANHATTAN|New York|     3101|         2|            Unknown|             Nenhuma|Instituti

In [3]:
from pyspark.sql.functions import count, countDistinct
gold_petroleo = tabela_petroleo \
    .groupBy("Contry", "Contributing_Factor")\
    .agg(
        count(tabela_petroleo.Spill_Number).alias("Derrames_Count"),
    ) \
   
gold_petroleo.show(50)

+--------------------+--------------------+--------------+
|              Contry| Contributing_Factor|Derrames_Count|
+--------------------+--------------------+--------------+
|              Broome|Missing Code in O...|             5|
|             Madison|             Unknown|           366|
|            Richmond|         Human Error|           335|
|            Cortland|   Equipment Failure|           463|
|            Allegany|     Abandoned Drums|            33|
|                Erie|             Unknown|          3413|
|            Tompkins|        Tank Failure|           167|
|              Cayuga|          Deliberate|           169|
|          Chautauqua|   Tank Test Failure|            83|
|            Cortland|          Deliberate|           104|
|              Oswego|               Other|           397|
|            Cortland|           Vandalism|             8|
|              Oswego|     Abandoned Drums|            78|
|            Chenango|    Traffic Accident|           16

In [4]:
# create air_quality table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto_gold.Tabela_Petroleo_2
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Petroleo_2 (
        Contry VARCHAR(100),
        Contributing_Factor VARCHAR(250),
        Derrames_Count LONG
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Petroleo_2/'
    """
)
        

DataFrame[]

In [5]:
# write to delta table
gold_petroleo \
    .write  \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Petroleo_2/")

In [6]:
# check the results in the table
spark.table("Projeto_gold.Tabela_Petroleo_2").show(50)


+--------------------+--------------------+--------------+
|              Contry| Contributing_Factor|Derrames_Count|
+--------------------+--------------------+--------------+
|              Broome|Missing Code in O...|             5|
|             Madison|             Unknown|           366|
|            Richmond|         Human Error|           335|
|            Cortland|   Equipment Failure|           463|
|            Allegany|     Abandoned Drums|            33|
|                Erie|             Unknown|          3413|
|            Tompkins|        Tank Failure|           167|
|              Cayuga|          Deliberate|           169|
|          Chautauqua|   Tank Test Failure|            83|
|            Cortland|          Deliberate|           104|
|              Oswego|               Other|           397|
|            Cortland|           Vandalism|             8|
|              Oswego|     Abandoned Drums|            78|
|            Chenango|    Traffic Accident|           16

In [9]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Petroleo_2/`
""").show()

++
||
++
++



In [10]:
spark.sql("""
DROP TABLE IF EXISTS Projeto_gold.Tabela_Petroleo2_Presto 
""").show()

spark.sql("""
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Petroleo2_Presto (
        Contry VARCHAR(100),
        Contributing_Factor VARCHAR(250),
        Derrames_Count LONG
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Petroleo_2/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++

